In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import gc
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
import warnings

import pandas as pd
import numpy as np

In [2]:
root_dir = "/kaggle/input/planets-dataset/planet/planet"
train_dir = os.path.join(root_dir, "train-jpg")
test_dir = os.path.join(root_dir, "test-jpg")

In [3]:
images_list = []
images = os.listdir(train_dir)

for image in images:
    img = load_img(os.path.join(train_dir, image), target_size=(64, 64))
    img = img_to_array(img, dtype="uint8")
    images_list.append(img)

In [4]:
images_list[0].shape

In [4]:
gc.collect()

In [5]:
train_data = np.asarray(images_list).astype(np.float16)

In [6]:
gc.collect()

In [7]:
labels_df = pd.read_csv(os.path.join(root_dir, "train_classes.csv"))
labels_df.head()

In [8]:
train_labels = labels_df.copy()

label_list = []
for tag in train_labels.tags.values:
    labels = tag.split(" ")
    for label in labels:
        if label not in label_list:
            label_list.append(label)
            
label_list.sort()
print(label_list)
print(len(label_list))

In [9]:
train_labels.head()

In [10]:
for label in label_list:
    train_labels[label] = train_labels["tags"].apply(lambda x: 1 if label in x.split(" ") else 0)
train_labels["image_name"] = train_labels["image_name"].apply(lambda x: x+".jpg")
    
train_labels.head()

In [11]:
labels_count = train_labels[label_list].sum().sort_values(ascending=False)
labels_count.plot.bar()

In [12]:
targets = []
for i in range(len(train_labels)):
    target = train_labels.iloc[i][2:].values
    targets.append(target)
    
print(len(targets))
training_labels = np.asarray(targets).astype(np.uint8)

In [13]:
gc.collect()

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_data, training_labels, test_size=0.2, shuffle=True, random_state=1)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)

In [43]:
test_csv = pd.read_csv(os.path.join(root_dir, "sample_submission.csv"))
test_csv.head()

In [44]:
submission = test_csv.copy()

In [45]:
submission["image_name"] = submission["image_name"].apply(lambda x: f"{x}.jpg")
submission.head()

In [87]:
test_names_df = submission.iloc[:]["image_name"].reset_index().drop("index", axis=1)
test_names_df.head()

In [89]:
test_names_df.shape

In [18]:
test_list = []
test_images = os.listdir(test_dir)

for image in test_images:
    img = load_img(os.path.join(test_dir, image), target_size=(64, 64))
    img = img_to_array(img, dtype="uint8")
    test_list.append(img)

In [21]:
test_list[0].shape

In [29]:
assert os.path.exists(test_images2)

In [19]:
test_images2 = os.listdir("../input/planets-dataset/test-jpg-additional/test-jpg-additional/")

for image in test_images2:
    img = load_img(os.path.join("../input/planets-dataset/test-jpg-additional/test-jpg-additional/", image), target_size=(64, 64))
    img = img_to_array(img, dtype="uint8")
    test_list.append(img)

In [23]:
len(test_list)

In [34]:
X_test = np.asarray(test_list).astype(np.float16)

In [20]:
gc.collect()

In [21]:
# Image generator
training_images = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255, validation_split=0.2)

train_images = training_images.flow_from_dataframe(dataframe=train_labels,
                                                   directory = train_dir,
                                                   x_col="image_name", y_col=label_list,
                                                   target_size=(64, 64),
                                                   class_mode="raw", seed=0,
                                                   batch_size=128, subset="training")

In [22]:
val_images = training_images.flow_from_dataframe(dataframe=train_labels,
                                                   directory = train_dir,
                                                   x_col="image_name", y_col=label_list,
                                                   target_size=(64, 64),
                                                   class_mode="raw", seed=0,
                                                   batch_size=128, subset="validation")

In [23]:
step_train_size = int(np.ceil(train_images.samples/train_images.batch_size))
step_val_size = int(np.ceil(val_images.samples/val_images.batch_size))
print(step_train_size)
print(step_val_size)

In [24]:
gc.collect()

In [26]:
def fbeta(ytrue , ypred, beta=2, epsilon=1e-4):
    beta_squarred = beta**2

    ytrue = tf.cast(ytrue, tf.float32)
    ypred = tf.cast(tf.greater(tf.cast(ypred, tf.float32), tf.constant(0.5)), tf.float32)
        
    tp = tf.reduce_sum(ytrue * ypred, axis=1)
    fp = tf.reduce_sum(ypred, axis=1) - tp
    fn = tf.reduce_sum(ytrue, axis=1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squarred)*precision*recall / (beta_squarred*precision + recall + epsilon)
    return fb

def multi_label_acc(ytrue , ypred, epsilon=1e-4):
    
    ytrue = tf.cast(ytrue, tf.float32)
    ypred = tf.cast(tf.greater(tf.cast(ypred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(ytrue * ypred, axis=1)
    fp = tf.reduce_sum(ypred, axis=1) - tp
    fn = tf.reduce_sum(ytrue, axis=1) - tp
    
    ytrue = tf.cast(ytrue, tf.bool)
    ypred = tf.cast(ypred, tf.bool)
    
    tn = tf.reduce_sum(tf.cast(tf.logical_not(ytrue), tf.float32) * tf.cast(tf.logical_not(ypred), tf.float32),\
                       axis=1)
    
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [27]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, Rescaling
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

num_classes = 17

model = Sequential()
model.add(Rescaling(1/255))
model.add(Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=[64, 64, 3], padding="same"))
model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))

model.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D((2,2), strides=2, padding="same"))

model.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D((2,2), strides=2, padding="same"))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))
model.compile(optimizer="adam",
              loss="binary_crossentropy",
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[multi_label_acc, fbeta])

In [28]:
# filepath = os.path.join(root_dir, "amazon_model.hdf5")
checkpoint = ModelCheckpoint(filepath="amazon_model.hdf5", monitor="val_fbeta", mode="max",
                                       save_best_only=True, save_weights_only=True)

In [29]:
gc.collect()

In [34]:
model.summary()

In [31]:
# Fitting the model
model.fit(x=X_train, y=y_train,
          steps_per_epoch=step_train_size,
          validation_data=(X_val, y_val),
          validation_steps=step_val_size,
          epochs=20, callbacks=[checkpoint])

In [32]:
gc.collect()

In [35]:
y_pred = model.predict(X_test, batch_size=24, verbose=1)

In [36]:
gc.collect()

In [38]:
y_pred[0]

In [90]:
result = pd.DataFrame(y_pred, columns=label_list)
result.head()

In [91]:
gc.collect()

In [92]:
predicted_res = []
for i in range(result.shape[0]):
    row = result.loc[[i]]
    row = row.apply(lambda x: x > 0.2, axis=1)
    row = row.transpose()
    row = row.loc[row[i] == True]
    " ".join(list(row.index))
    predicted_res.append(" ".join(list(row.index)))

In [93]:
test_names_df["image_name"] = test_names_df["image_name"].apply(lambda x: x[:-4])
    
test_names_df.loc[40482]

In [99]:
test_names_df["tags"] = predicted_res
test_names_df.to_csv("my_submission2.csv", index=False)